In [1]:
import pandas as pd
import numpy as np
# import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.model_selection import train_test_split
import json
import re
import tensorflow as tf
import numpy as np
import more_itertools
import json
from neo4j import GraphDatabase, Driver
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
# %matplotlib inline

In [2]:
candidate = pd.read_excel("candidate_master_data.xlsx",sheet_name='Candidate Data')
candidate.to_csv("candidate_master.csv",index=None,header=True)
cand_df = pd.read_csv("candidate_master.csv")
# cand_df.head()

In [3]:
def skill_parsing(skill):
    '''
    Function that extracts the skills from the Skill column 
    '''
    res = json.loads(skill,strict=False)
    skill_list=res['skills']
    
    skills = []
    
    for i in range (len(skill_list)):
        if skill_list[i]['skill_type'] == 'primary':
            skills.append(skill_list[i]['skill'])
        else:
            if skill_list[i]['skill_type'] == 'secondary':
                skills.append(skill_list[i]['skill'])
    return skills

can_skills ={}
skill_dict = dict(zip(cand_df['Candidate ID'],cand_df['Skills']))

for i in range (len(skill_dict)):
    for k,v in skill_dict.items():
        values=skill_dict[k]
        skill_set=skill_parsing(values)
        can_skills.update({k:skill_set})

In [4]:
requisition = pd.read_excel("requisition_data.xlsx",sheet_name='Requisition Data')
requisition.to_csv("requisition.csv",index=None,header=True)
req_df = pd.read_csv("requisition.csv")
req_df = req_df[req_df['Skills'].notna()]
# req_df

In [5]:
req_skills = []
for i in range (len(req_df)):
    text = req_df['Primary Skills (Extracted)'][i]
    req_skills.append(re.split('\n,|,',text))

In [6]:
req_dict = dict(zip(req_df['Requisition ID'],req_skills))

In [7]:
final_df=cand_df[['Candidate ID','Current Designation','Total Experience ( in Months)', 'Education__degree']]
final_df.drop_duplicates('Candidate ID',inplace=True)

C:\Users\gopinath.selvaraj\Anaconda3\envs\sample\lib\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [8]:
can_keys=can_skills.keys()
can_values=can_skills.values()

In [9]:
new_list=[]
str1=' '
for i in range (len(can_values)):
    s=str1.join(list(can_values)[i])
    new_list.append(s)
    str1=' '

data={'Candidate ID':can_keys, 'Can_Skills':new_list}
skill_df = pd.DataFrame.from_dict(data)

In [10]:
final_df=final_df.merge(skill_df,left_on=['Candidate ID'], right_on=['Candidate ID'],how='left')

In [11]:
def query(req_skill, pri_skill, arg):
    query = """
            MATCH
            (baseSkill:Skill)-[eq_rel:EQUIVALENT]->(adj_Skill:AdjacentSkill)
            where baseSkill.name in $req_skill 
            and adj_Skill.name in $pri_skill
            return 'AdjacentSkillScore' as ScoreType ,SUM(toInteger(eq_rel.value)) as Scores
            UNION
            MATCH
            (baseSkill:Skill)-[spec_rel:SPECIFIC_TO]->(subSkill:Subskill)
            where baseSkill.name in $req_skill 
            and subSkill.name in $pri_skill
            return 'SubSkillScores' as ScoreType , SUM(toInteger(spec_rel.value)) as Scores
            UNION
            MATCH
            (baseSkill:Skill)-[impe_rel:IMPLEMENTED_WITH]->(compositeSkill:Composite)
            where baseSkill.name in $req_skill  
            and compositeSkill.name in $pri_skill
            return 'CompositeSkillScore' as ScoreType, SUM(toInteger(impe_rel.value)) as Scores

           """
    query_params = {
                "req_skill": ["javascript"],
                "pri_skill": pri_skill
                }

    with open('./settings.json') as f:
        settings = json.load(f)[arg]

    driver = GraphDatabase.driver(
            settings["neo4j_url"], 
            auth=(settings["neo4j_user"], settings["neo4j_password"]),
            encrypted=False)

    with driver.session() as session:
        data = session.run(query,**query_params).data()
    return data
    

In [12]:
tot_score=0
score_dict = {}
for k,v in can_skills.items():
    pri_skill=v
    query_result=query(req_skill='python',pri_skill=pri_skill,arg='local')
    for i in range (len(query_result)):
        tot_score=tot_score+query_result[i]['Scores']
    score_dict[k]=tot_score
    tot_score=0

In [13]:
scr_keys=score_dict.keys()
scr_values=score_dict.values()
data={'Candidate ID':scr_keys, 'Score':scr_values}

In [14]:
score_df = pd.DataFrame.from_dict(data)

In [15]:
final_df=final_df.merge(score_df,left_on=['Candidate ID'], right_on=['Candidate ID'],how='left')
final_df

,Candidate ID,Current Designation,Total Experience ( in Months),Education__degree,Can_Skills,Score
0,1000114023,Associate IT Consultant,36.0,B.Tech,c java html javascript jquery python servlets ...,2
1,1000236646,SAP ABAP Consultant,36.0,NaN,sap implementation technical developments requ...,0
2,1000237724,Technical Solution Architect,127.0,NaN,java j2ee c c + + servlets jsp jsf spring hibe...,1
3,1000243635,NaN,54.0,Bachelor of Engineering,backend processes fte reduction business requi...,1
4,1000291737,Project Lead,180.0,Master in Computer Application,agile methodology scrum kanban deep learning j...,1
...,...,...,...,...,...,...
102,1001660345,NaN,264.0,Executive General Management Program,business planning global delivery model scope ...,0
103,1001970790,Lead Manager,360.0,Bachelor,psm project management production support envi...,0
104,1001428388,Associate Cyber Security Consultant,34.0,B.E,application penetration testing php html . net...,2
105,1001453068,Software Engineer,36.0,Bachelor of Science,javascript react native html css sql problem s...,1


In [16]:
final_df.isna().sum()

Candidate ID                      0
Current Designation              19
Total Experience ( in Months)     0
Education__degree                13
Can_Skills                        0
Score                             0
dtype: int64

In [17]:
# Check for columns which arent numbers
for label, content in final_df.items():
    if not pd.api.types.is_numeric_dtype(content):
        print(label)

Current Designation
Education__degree
Can_Skills


In [18]:
# Turn categorical variables into numbers and fill them
for label, content in final_df.items():
    if not pd.api.types.is_numeric_dtype(content):
#         final_df[label+"_is_missing"] = pd.isnull(content)
        final_df[label] = pd.Categorical(content).codes + 1

In [19]:
# Find the columns that contains strings
for label, content in final_df.items():
    if pd.api.types.is_string_dtype(content):
        print(label)

In [20]:
# This will change all string values to categories
for label, content in final_df.items():
    if pd.api.types.is_string_dtype(content):
        final_df[label] = content.astype("category").cat.as_ordered()

In [21]:
final_df.isna().sum()

Candidate ID                     0
Current Designation              0
Total Experience ( in Months)    0
Education__degree                0
Can_Skills                       0
Score                            0
dtype: int64

In [22]:
final_df

,Candidate ID,Current Designation,Total Experience ( in Months),Education__degree,Can_Skills,Score
0,1000114023,8,36.0,8,19,2
1,1000236646,37,36.0,0,85,0
2,1000237724,71,127.0,0,45,1
3,1000243635,0,54.0,17,14,1
4,1000291737,31,180.0,31,2,1
...,...,...,...,...,...,...
102,1001660345,0,264.0,24,15,0
103,1001970790,25,360.0,13,71,0
104,1001428388,6,34.0,5,7,2
105,1001453068,62,36.0,19,55,1


In [23]:
X = final_df[['Candidate ID','Current Designation','Total Experience ( in Months)','Education__degree','Can_Skills']]
y = final_df['Score']

In [24]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

In [25]:
X_train_d = X_train
X_train_d = X_train_d.drop(['Candidate ID'],axis=1)
X_test_d = X_test
X_test_d = X_test_d.drop(['Candidate ID'],axis=1)

In [26]:
from sklearn.preprocessing import MinMaxScaler
scalar = MinMaxScaler()
scalar.fit(X_train_d)
X_trans = scalar.transform(X_train_d)

In [27]:
def generate_model():
    model = Sequential([
        Dense(32,input_shape=(4,),activation='relu'),
        Dense(16, activation='relu'),
        Dense(12, activation='relu'),
        Dense(1, activation='relu'),
    ])

    return model

In [28]:
generate_model()

In [29]:
model = generate_model()
model.compile(loss='mean_squared_error',
                optimizer='adam',
                metrics=['accuracy'])

In [30]:
output=model.fit(X_trans,y_train,batch_size=10, epochs=200)

Epoch 1/200
8/8 [==============================] - 1s 2ms/step - loss: 1.3734 - accuracy: 0.5135
Epoch 2/200
8/8 [==============================] - 0s 1ms/step - loss: 1.2408 - accuracy: 0.5000
Epoch 3/200
8/8 [==============================] - 0s 2ms/step - loss: 1.1670 - accuracy: 0.5270
Epoch 4/200
8/8 [==============================] - 0s 1ms/step - loss: 1.1302 - accuracy: 0.4324
Epoch 5/200
8/8 [==============================] - 0s 1ms/step - loss: 1.1115 - accuracy: 0.4189
Epoch 6/200
8/8 [==============================] - 0s 2ms/step - loss: 1.1102 - accuracy: 0.4189
Epoch 7/200
8/8 [==============================] - 0s 1ms/step - loss: 1.1024 - accuracy: 0.4189
Epoch 8/200
8/8 [==============================] - 0s 2ms/step - loss: 1.0893 - accuracy: 0.4459
Epoch 9/200
8/8 [==============================] - 0s 1ms/step - loss: 1.0842 - accuracy: 0.4324
Epoch 10/200
8/8 [==============================] - 0s 1ms/step - loss: 1.0783 - accuracy: 0.4459
Epoch 11/200
8/8 [===========

8/8 [==============================] - 0s 2ms/step - loss: 0.7794 - accuracy: 0.6216
Epoch 85/200
8/8 [==============================] - 0s 2ms/step - loss: 0.7772 - accuracy: 0.6081
Epoch 86/200
8/8 [==============================] - 0s 2ms/step - loss: 0.7759 - accuracy: 0.6216
Epoch 87/200
8/8 [==============================] - 0s 2ms/step - loss: 0.7765 - accuracy: 0.6081
Epoch 88/200
8/8 [==============================] - 0s 2ms/step - loss: 0.7761 - accuracy: 0.6216
Epoch 89/200
8/8 [==============================] - 0s 2ms/step - loss: 0.7710 - accuracy: 0.6351
Epoch 90/200
8/8 [==============================] - 0s 2ms/step - loss: 0.7767 - accuracy: 0.6351
Epoch 91/200
8/8 [==============================] - 0s 2ms/step - loss: 0.7696 - accuracy: 0.5946
Epoch 92/200
8/8 [==============================] - 0s 2ms/step - loss: 0.7676 - accuracy: 0.5946
Epoch 93/200
8/8 [==============================] - 0s 2ms/step - loss: 0.7652 - accuracy: 0.6351
Epoch 94/200
8/8 [===============

8/8 [==============================] - 0s 2ms/step - loss: 0.6468 - accuracy: 0.6892
Epoch 168/200
8/8 [==============================] - 0s 2ms/step - loss: 0.6240 - accuracy: 0.7568
Epoch 169/200
8/8 [==============================] - 0s 2ms/step - loss: 0.6156 - accuracy: 0.7432
Epoch 170/200
8/8 [==============================] - 0s 2ms/step - loss: 0.6089 - accuracy: 0.6892
Epoch 171/200
8/8 [==============================] - 0s 2ms/step - loss: 0.6105 - accuracy: 0.6757
Epoch 172/200
8/8 [==============================] - 0s 2ms/step - loss: 0.6016 - accuracy: 0.6892
Epoch 173/200
8/8 [==============================] - 0s 2ms/step - loss: 0.6053 - accuracy: 0.6757
Epoch 174/200
8/8 [==============================] - 0s 2ms/step - loss: 0.6029 - accuracy: 0.7162
Epoch 175/200
8/8 [==============================] - 0s 2ms/step - loss: 0.5998 - accuracy: 0.7027
Epoch 176/200
8/8 [==============================] - 0s 2ms/step - loss: 0.5968 - accuracy: 0.7027
Epoch 177/200
8/8 [=====

In [31]:
print(f"Accuracy: {round(output.history['accuracy'][-1]*100)}%")

Accuracy: 69%


In [32]:
# %%time
# from sklearn.ensemble import RandomForestRegressor

# model_RFR = RandomForestRegressor(n_jobs=-1)

# model_RFR.fit(X_train,y_train)

In [33]:
# from sklearn.metrics import mean_squared_log_error, mean_absolute_error

# def rmsle(y_test,y_preds):
#     return np.sqrt(mean_squared_log_error(y_test,y_preds))

# def show_scores(model_RFR):
#     train_preds = model_RFR.predict(X_train)
#     scores = {"Training MAE": mean_absolute_error(y_train,train_preds),
#               "Training RMSLE": rmsle(y_train,train_preds)}
    
#     return scores

In [34]:
Xnew = scalar.transform(X_test_d)
# make a prediction
ynew = model.predict(Xnew)
# for i in range (len(ynew)):
#     print(ynew[i])
#     ynew[i]=ynew[i].round()
# show the inputs and predicted outputs
# for i in range(len(Xnew)):
# 	print("Candidate ID=%s, Predicted Score=%s" % (Xnew[i], ynew[i]))


In [35]:
y_score = []
for i in range (len(ynew)):
    y_score.append(ynew[i][0])

In [36]:
final_score=dict(zip(X_test['Candidate ID'],y_score))

In [37]:
for k,v in final_score.items():
    print(f'The Candidate {k} has a score of {v}')

The Candidate 1001811121 has a score of 1.3572280406951904
The Candidate 1000667374 has a score of 0.0
The Candidate 1000291737 has a score of 0.0
The Candidate 1001578189 has a score of 0.0
The Candidate 1001535281 has a score of 0.48448625206947327
The Candidate 1001524122 has a score of 0.0798758938908577
The Candidate 1001427892 has a score of 1.0098786354064941
The Candidate 1001454646 has a score of 1.4671205282211304
The Candidate 1000986309 has a score of 1.4316527843475342
The Candidate 1000777836 has a score of 0.7764728665351868
The Candidate 1001838419 has a score of 0.312822163105011
The Candidate 1001467897 has a score of 1.1230062246322632
The Candidate 1000114023 has a score of 1.9593079090118408
The Candidate 1001841272 has a score of 0.4565700590610504
The Candidate 1001353550 has a score of 0.0
The Candidate 1001453068 has a score of 0.5394503474235535
The Candidate 1001484939 has a score of 0.6822609305381775
The Candidate 1001819038 has a score of 0.833901703357696

In [38]:
data1 = {'Candidate ID':final_score.keys(), 'Score':final_score.values()}
score_dataframe = pd.DataFrame.from_dict(data1)

In [39]:
score_dataframe['Rank']=score_dataframe['Score'].rank(method='max',ascending=False)
score_dataframe=score_dataframe.sort_values(by='Rank')
score_dataframe

,Candidate ID,Score,Rank
27,1001668286,2.259871,1.0
12,1000114023,1.959308,2.0
24,1001372477,1.689189,3.0
21,1001441754,1.484034,4.0
7,1001454646,1.467121,5.0
8,1000986309,1.431653,6.0
0,1001811121,1.357228,7.0
26,1001484944,1.246752,8.0
25,1001431743,1.240517,9.0
11,1001467897,1.123006,10.0


In [44]:
# save the model to disk
model.save('final_model.h5')